# Construcción del dash

* El dashboard está dividido en dos secciones principales: la sección de **navegación** a la izquierda y la sección de **contenido** a la derecha.

* **Sección de navegación:**
Esta sección, ubicada a la izquierda, contiene tres botones interactivos que permiten cambiar la visualización de la información en la sección de contenido. Los botones están diseñados con una tipografía clara y legible, como Arial o Helvetica, y están espaciados uniformemente para facilitar la navegación. Cada botón tiene un título descriptivo correspondiente a su función: "Incio", "Análisis exploratorio de datos (EDA)" y "Modelos predictivos".

* **Sección de contenido:**
Esta sección, ubicada a la derecha, muestra la información correspondiente al botón seleccionado en la sección de navegación.

    1. **Inicio:** Esta ventana proporciona una visión general del conjunto de datos.

    2. **EDA:** Esta ventana presenta un análisis de los datos. Incluye gráficos de la serie de tiempo en diversas escalas temporales (mensual, anual, decenal), gráficos de línea, gráficos de caja y bigotes, mapas de calor, proporcionando información sobre patrones y tendencias de la variable objetivo (Anomalía de temperatura (°C)) de forma interactiva. Asimismo, se explora la estacionariedad de la serie, información base para la construcción de modelos predictivos.

    3. **Modelos predictivos:** Aquí presenta los resultados de tres modelos predictivos: ARIMA, suavización exponencial y regresión polinómica. Cada modelo se presenta en un gráfico separado que muestra los datos históricos y las predicciones del modelo. También se proporcionan medidas de precisión del modelo, como el error cuadrático medio (MSE) y el coeficiente de determinación (R^2).


```Python

import dash
from dash import html, dcc
from dash import dash_table
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tools.eval_measures import aic
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from dash.dependencies import Input, Output, State
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import plotly.express as px
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
import pickle

ordered_months = ['january', 'february', 'march', 'april', 'may', 'june', 
                  'july', 'august', 'september', 'october', 'november', 'december']

# Cargamos el conjunto de datos
timeseries_URL = "https://raw.githubusercontent.com/SandraMaldonado19/Dash_PF_Dataviz/main/df_timeseries.csv"
df_tseries = pd.read_csv(timeseries_URL)


app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Análisis de Anomalía de Temperatura (°C)"),
    dcc.Dropdown(
        id='module-dropdown',
        options=[
            {'label': 'Sobre los datos', 'value': 'data'},
            {'label': 'EDA', 'value': 'eda'},
            {'label': 'Modelo predictivo', 'value': 'predictive'}
        ],
        value='data'
    ),
    html.Div(id='module-content')
])

@app.callback(
    Output('module-content', 'children'),
    Input('module-dropdown', 'value'))
def update_module_content(selected_module):
    if selected_module == 'data':
        return html.Div([
            dcc.Tabs([
                dcc.Tab(label='General', children=[
                    html.Div(id='general-info')
                ]),
                dcc.Tab(label='Visualización de Datos', children=[
                    html.Div(id='data-table')
                ]),
            ])
        ])
    elif selected_module == 'eda':
        return html.Div([
            dcc.Tabs([
                dcc.Tab(label='Serie de Tiempo', children=[
                    dcc.Graph(id='time-series')
                ]),
                dcc.Tab(label='Boxplot', children=[
                    dcc.Dropdown(
                        id='plot-type-dropdown',
                        options=[
                            {'label': 'Por Mes', 'value': 'month'},
                            {'label': 'Por Año', 'value': 'year'}
                        ],
                        value='month'
                    ),
                    dcc.Graph(id='boxplot')
                ]),
                dcc.Tab(label='Heatmap', children=[
                    dcc.Graph(id='heatmap')
                ]),
                dcc.Tab(label='Gráfico de Barras', children=[
                    dcc.Graph(id='bar-chart')
                ]),
                dcc.Tab(label='Comparación de Años', children=[
                     dcc.Graph(id='temperature-graph')
                   ]),
                dcc.Tab(label='Autocorrelación', children=[
                    dcc.Graph(id='autocorrelation'),
                    dcc.Graph(id='partial-autocorrelation')
                ]),
            ])
        ])
        
    elif selected_module == 'predictive':
        return html.Div([
            dcc.Tabs([
                dcc.Tab(label='Modelo ARIMA', children=[
                    html.Button('Realizar Predicción ARIMA', id='arima-button'),
                    html.Div(id='arima-prediction'),
                    dcc.Graph(id='arima-plot'),
                    html.Div(id='arima-residuals')
                ]),
                dcc.Tab(label='Suavización Exponencial', children=[
                    html.Button('Realizar Predicción Suavización Exponencial', id='exp-smooth-button'),
                    html.Div(id='exp-smooth-prediction'),
                    dcc.Graph(id='exp-smooth-plot')
                ]),
                dcc.Tab(label='Polinomial', children=[
                    html.Button('Realizar Predicción Polinomial', id='polinomial-button'),
                    html.Div(id='polinomial-prediction'),
                ]),
            ])
        ])
  
@app.callback(
    Output('general-info', 'children'),
    Input('module-dropdown', 'value'))
def update_general_info(selected_module):
    if selected_module == 'data':
        return dcc.Markdown('''
        Presentamos el análisis de un conjunto de datos llamado HadCRUT5, que mide los cambios de temperatura cercana a la superficie en todo el mundo, entre 1850 a 2018. HadCRUT5 presenta anomalías mensuales promedio de temperatura cercana a la superficie, en relación con el período 1961–1990. En general, este conjunto de datos muestra un aumento del calentamiento promedio global desde mediados del siglo 19 y en los últimos años, en consonancia con otros análisis. Este fenómeno se debe a múltiples factores, incluidos una mejor representación del calentamiento del Ártico y una mejor comprensión de los sesgos en evolución en las mediciones sobre la superficie del mar y la tierra.
        ''')

@app.callback(
    Output('data-table', 'children'),
    Input('module-dropdown', 'value'))
def update_data_table(selected_module):
    if selected_module == 'data':
        return dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df_tseries.columns],
            data=df_tseries.to_dict('records'),
        )

df_tseries['month'] = df_tseries['month'].astype('category')
df_tseries['year'] = df_tseries['year'].astype('category')

# Función para calcular la media móvil y el intervalo de confianza
def calculate_rolling_mean(df, window_size=12, num_of_std=1.96):
    df['rolling_mean'] = df['value'].rolling(window=window_size).mean()
    df['rolling_std'] = df['value'].rolling(window=window_size).std()
    df['upper_band'] = df['rolling_mean'] + (df['rolling_std'] * num_of_std)
    df['lower_band'] = df['rolling_mean'] - (df['rolling_std'] * num_of_std)
    return df


@app.callback(
    Output('time-series', 'figure'),
    Input('module-dropdown', 'value'))
def update_time_series(selected_module):
    # Calcular la media móvil y el intervalo de confianza
    df_timeseries = calculate_rolling_mean(df_tseries.copy())  # Asegúrate de tener una copia del DataFrame original

    # Crear la figura base con la serie de tiempo original
    fig = px.line(df_timeseries, x='date', y='value', labels={'value': 'Serie de tiempo'}, color_discrete_sequence=['blue'])

    # Agregar la media móvil
    fig.add_trace(go.Scatter(x=df_timeseries['date'], y=df_timeseries['rolling_mean'], mode='lines', name='Media móvil', line=dict(color='red')))

    # Agregar el intervalo de confianza
    fig.add_trace(go.Scatter(x=df_timeseries['date'], y=df_timeseries['upper_band'], mode='lines', name='IC - 95% superior', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=df_timeseries['date'], y=df_timeseries['lower_band'], mode='lines', name='IC - 95% inferior', line=dict(color='green')))

    # Marcar la media correspondiente al año cuando se pasa el ratón sobre la gráfica
    if isinstance(selected_module, dict) and 'points' in selected_module:
        year_hovered = selected_module['points'][0]['x'].split('-')[0]
        mean_value = df_timeseries[df_timeseries['year'] == int(year_hovered)]['rolling_mean'].iloc[0]
        fig.add_trace(go.Scatter(x=[selected_module['points'][0]['x']], y=[mean_value], mode='markers', name=f'Media {year_hovered}', marker=dict(color='black')))

    fig.update_layout(
        xaxis_title='Fecha',
        yaxis_title='Anomalía de temperatura (°C)',
        margin={'b': 30, 'r': 10, 'l': 60, 't': 50},
        legend={'x': 0, 'y': 1}
    )

    return fig



@app.callback(
    Output('boxplot', 'figure'),
    Input('plot-type-dropdown', 'value'))
def update_boxplot(selected_plot_type):
    global df_tseries
    ordered_months = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
    if selected_plot_type == 'month':
        fig = px.box(df_tseries, x='month', y='value', color='month',
                     labels={'month': 'Mes', 'value': 'Anomalía de Temperatura (°C)'},
                     category_orders={'month': ordered_months})
    elif selected_plot_type == 'year':
        fig = px.box(df_tseries, x='year', y='value', color='year',
                     labels={'year': 'Año', 'value': 'Anomalía de Temperatura (°C)'})
    else:
        fig = go.Figure()
    return fig
  

@app.callback(
    Output('heatmap', 'figure'),
    Input('module-dropdown', 'value'))
def update_heatmap(selected_module):
    if selected_module == 'eda':
        heatmap_data = df_tseries.groupby(['month', 'year']).value.mean().unstack()
        heatmap_data = heatmap_data.reindex(ordered_months)

        fig = go.Figure(data=go.Heatmap(
            z=heatmap_data.values,
            x=heatmap_data.columns,
            y=heatmap_data.index,
            colorscale='RdBu',
            reversescale=True
        ))

        fig.update_layout(
            xaxis_title='Año',
            yaxis_title='Mes'
        )

        return fig
  

@app.callback(
    Output('bar-chart', 'figure'),
    Input('module-dropdown', 'value'))
def update_bar_chart(selected_module):
    if selected_module == 'eda':
        df_tseries['year'] = df_tseries['year'].astype(int)  # Convertir 'year' a int
        df_tseries['decade'] = (df_tseries['year'] // 10) * 10
        decade_means = df_tseries.groupby('decade')['value'].mean().reset_index()

        fig = px.bar(decade_means, x='decade', y='value', color=(decade_means['value'] > 0),
                     labels={'decade': 'Década', 'value': 'Anomalía de Temperatura (°C)'})

        fig.update_layout(showlegend=False)  # Ocultar la leyenda

        return fig
  


# Callback año máximo
@app.callback(
    Output('temperature-graph', 'figure'),
    [Input('year-picker', 'value')]
)
def update_figure(selected_years):
    traces = []
    markers = ['circle', 'square', 'diamond', 'cross', 'x']

    # Añade las líneas para los años más cálidos
    for i, year in enumerate(top_years):
        df_year = df_tseries[df_tseries['year'] == year]
        traces.append(go.Scatter(
            x=df_year['month'],
            y=df_year['value'],
            mode='lines+markers',
            name=str(year),
            marker=dict(
                symbol=markers[i % len(markers)],
                size=10
            )
        ))

@app.callback(
    Output('autocorrelation', 'figure'),
    Input('module-dropdown', 'value'))
def update_autocorrelation(selected_module):
    if selected_module == 'eda':
        # Crear una figura con subplots
        fig = make_subplots(rows=3, cols=2)

        # Agregar los gráficos a los subplots correspondientes
        fig.add_trace(go.Scatter(y=df_tseries.value, mode='lines', line=dict(color='blue'), showlegend=False, hovertemplate='Value: %{y}'), row=1, col=1)
        fig.add_trace(go.Scatter(y=df_tseries.value.diff(), mode='lines', line=dict(color='red'), showlegend=False, hovertemplate='Diff: %{y}'), row=2, col=1)
        fig.add_trace(go.Scatter(y=df_tseries.value.diff().diff(), mode='lines', line=dict(color='green'), showlegend=False, hovertemplate='Diff Diff: %{y}'), row=3, col=1)

        # Para agregar los gráficos de autocorrelación, se necesita calcular los valores de autocorrelación
        acf_values = acf(df_tseries.value, nlags=2080)
        fig.add_trace(go.Scatter(y=acf_values, mode='lines', fill='tozeroy', line=dict(color='blue'), showlegend=False, hovertemplate='ACF: %{y}'), row=1, col=2)

        diff1_values = df_tseries.value.diff()
        acf_diff1_values = acf(diff1_values.dropna(), nlags=2080)
        fig.add_trace(go.Scatter(y=acf_diff1_values, mode='lines', fill='tozeroy', line=dict(color='red'), showlegend=False, hovertemplate='ACF Diff: %{y}'), row=2, col=2)

        diff2_values = df_tseries.value.diff().diff()
        acf_diff2_values = acf(diff2_values.dropna(), nlags=2080)
        fig.add_trace(go.Scatter(y=acf_diff2_values, mode='lines', fill='tozeroy', line=dict(color='green'), showlegend=False, hovertemplate='ACF Diff Diff: %{y}'), row=3, col=2)


        # Agregar títulos a las gráficas

        fig.add_annotation(dict(text='Serie original', x=0.20, y=1.1, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
        fig.add_annotation(dict(text='1er Orden de diferenciación', x=0.15, y=0.66, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
        fig.add_annotation(dict(text='2do Orden de diferenciación', x=0.15, y=0.25, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))

        fig.add_annotation(dict(text='ACF de la Serie original', x=0.85, y=1.1, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
        fig.add_annotation(dict(text='ACF del 1er Orden de diferenciación', x=0.88, y=0.66, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))
        fig.add_annotation(dict(text='ACF del 2do Orden de diferenciación', x=0.88, y=0.25, xref='paper', yref='paper', showarrow=False, font=dict(size=14)))

        return fig



@app.callback(
    Output('partial-autocorrelation', 'figure'),
    Input('module-dropdown', 'value'))
def update_partial_autocorrelation(selected_module):
    if selected_module == 'eda':
        pacf_result = sm.tsa.pacf(df_tseries['value'], nlags=1040)

        fig = go.Figure(data=go.Scatter(x=list(range(len(pacf_result))), y=pacf_result, mode='lines'))
        fig.update_layout(xaxis_title='Lag', yaxis_title='Autocorrelación Parcial', title='Autocorrelación Parcial de la Serie Original')

        return fig
  

np.random.seed(123)  # Fijar semilla para reproducibilidad

# Ordenar los datos por fecha en orden ascendente
df_tseries = df_tseries.sort_values('date')


total_rows = len(df_tseries)
train_size = round(total_rows * 0.8)
test_size = total_rows - train_size

train = df_tseries.value.iloc[:train_size]
dates_train = df_tseries.date.iloc[:train_size]

test = df_tseries.value.iloc[train_size:train_size + test_size]
dates_test = df_tseries.date.iloc[train_size:train_size + test_size]

train_df = df_tseries[["date", "value"]].iloc[:train_size]
test_df = df_tseries[["date", "value"]].iloc[train_size:train_size + test_size]

best_order = (1, 1, 2)

@app.callback(
    Output('arima-prediction', 'children'),
    Input('arima-button', 'n_clicks'))
def update_arima_prediction(n_clicks):
    if n_clicks is not None:
        modelo_arima = ARIMA(train, order=(1,1,2)).fit()
        forecast_results = modelo_arima.forecast(test)
        forecast, stderr, conf_int = forecast_results
        # Crear un DataFrame con las predicciones
        df = pd.DataFrame({'Forecast': forecast, 'StdErr': stderr, 'ConfInt': conf_int})
            # Convertir el DataFrame a una tabla HTML
        table = df.to_html()
        return html.Div([
                html.H3('Resumen del modelo:'),
                html.P(modelo_arima.summary().as_html()),  # Convertir el resumen del modelo a HTML
                html.H3('Predicciones para los próximos {} períodos:'.format(len(forecast))),
                html.Div(table, style={'overflowX': 'auto'})  # Añadir desplazamiento horizontal si la tabla es demasiado ancha
            ])

test_tt = test.tolist()

def arima_rolling(history, test, best_order):
        predictions = list()
        for t in range(len(test)):
            model = ARIMA(history, order=best_order)
            model_fit = model.fit()
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test[t]
            history.append(obs)
            #print('predicted=%f, expected=%f' % (yhat, obs))
        return predictions


@app.callback(
    Output('arima-plot', 'figure'),
    Input('arima-button', 'n_clicks'))
def update_arima_plot(n_clicks):
    fig = px.line()
    if n_clicks is not None:
        best_model = ARIMA(train, order=(1, 1, 2)).fit()
        if best_model is not None:
            predicted_values = arima_rolling(train.tolist(), test_tt, best_order)
            fig.add_scatter(x=dates_train, y=train, mode='lines', name='Entrenamiento', line=dict(color='green'))
            fig.add_scatter(x=dates_test, y=test_tt, mode='lines', name='Prueba', line=dict(color='blue'))
            fig.add_scatter(x=dates_test, y=predicted_values, mode='lines', name='Pronóstico', line=dict(color='red'))

    return fig


@app.callback(
    Output('arima-residuals', 'children'),
     Input('arima-button', 'n_clicks'))
def update_arima_residuals(n_clicks):
    if n_clicks is not None:
        best_model = ARIMA(train, order=(1, 1, 2)).fit()
        if best_model is not None:
            residuals = best_model.resid
            ljung_box_results = acorr_ljungbox(residuals, lags=[10])
            return f'Ljung-Box test: {ljung_box_results[1][0]}'
  

@app.callback(
    Output('exp_smooth_prediction', 'children'),
    Input('exp_smooth_button', 'n_clicks'),
    )       
def update_exp_smooth_prediction(n_clicks):
    if n_clicks is not None:
        model = SimpleExpSmoothing(train)
        model_fit = model.fit()
        forecast_result = model_fit.forecast()
        return str(forecast_result)

@app.callback(
    Output('exp_smooth_plot', 'figure'),
    Input('exp_smooth_button', 'n_clicks'),
    State('exp_smooth_periods', 'value'))
def update_exp_smooth_plot(n_clicks):
    if n_clicks is not None:
        ses_model = SimpleExpSmoothing(df_tseries['value']).fit()
        forecast_result_ses = ses_model.forecast()

        fig = make_subplots()
        fig.add_trace(go.Scatter(y=forecast_result_ses, mode='lines', name='Predicción'))
        fig.update_layout(title='Modelo de Suavización Exponencial Simple y Pronósticos')

        return fig
  

def fit_polynomial(data, degree, periods):
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    X = np.arange(len(data)).reshape(-1, 1)
    y = data['value'].values
    model.fit(X, y)
    future_seq = np.arange(len(data), len(data) + periods).reshape(-1, 1)
    future_data = pd.DataFrame({'index': future_seq.flatten(), 'value': np.nan})
    future_data['predicted'] = model.predict(future_seq)
    return model, future_data


@app.callback(
    Output('polynomial_prediction', 'children'),
    Input('polynomial_button', 'n_clicks'),)
def update_polynomial_prediction(n_clicks):
    if n_clicks is not None:
        model, future_data = fit_polynomial(train, 3,15)
        return str(future_data['predicted'])
    
if __name__ == '__main__':
    #app.run_server(debug=True) # <- For testing purposes
    app.run_server(debug=True, host='0.0.0.0', port=9000) # <- To Dockerize the Dash

```